In [1]:
comp = dict({'0':'0101010','1':'0111111','-1':'0111010','D':'0001100','A':'0110000','M':'1110000',
            '!D':'0001101','!A':'0110001','!M':'1110001','-D':'0001111','-A':'0110011','-M':'1110011',
           'D+1':'0011111','A+1':'0110111','M+1':'1110111','D-1':'0001110','A-1':'0110010','M-1':'0110010',
           'D+A':'0000010','D+M':'1000010','A+D':'0000010','M+D':'1000010','D-A':'0010011','D-M':'1010011',
             'A-D':'0000111','M-D':'1000111','D&A':'0000000','D&M':'1000000','D|A':'0010101','D|M':'1010101'})

dest = dict({'null':'000','M':'001','D':'010','MD':'011','A':'100','AM':'101','AD':'110','AMD':'111'})

jump = dict({'null':'000','JGT':'001','JEQ':'010','JGE':'011','JLT':'100','JNE':'101','JLE':'110','JMP':'111'})

In [2]:
a_instruction = dict({'R0':'0','R1':'1','R2':'2','R3':'3','R4':'4','R5':'5','R6':'6','R7':'7','R8':'8','R9':'9','R10':'10',
                      'R11':'11','R12':'12','R13':'13','R14':'14','R15':'15','SCREEN':'16384','KBD':'24576'})

In [3]:
original = open('mult.asm','r')
remove_comments = [row.split('//')[0].replace(" ","").replace("\n","") for row in original]
clean_blank_lines = [row for row in remove_comments if row != ""]
labels = [row.replace("(","").replace(")","") for row in clean_blank_lines if row[0]=='(' and row[-1]==')']

In [4]:
labels_dict = dict()
count_labels = 0
for label in labels:
    for row in clean_blank_lines:
        clean_row = row.replace("(","").replace(")","")
        if clean_row == label:
            key = label.replace("(","").replace(")","")
            labels_dict[key] = clean_blank_lines.index(row) - count_labels
            count_labels += 1

In [5]:
remove_labels = [row for row in clean_blank_lines if row[0]!='(' and row[-1]!=')']
subst_a_inst = []
for item in remove_labels:
    if item[0] == '@':
        new_item = ''
        if item[1:] in labels:
            new_item = '@' + str(labels_dict[item[1:]])
        else:
            new_item = '@' + a_instruction[item[1:]]
        if new_item == '':
            print("Error founded")
            break
        subst_a_inst.append(new_item)
    else:
        subst_a_inst.append(item)

In [6]:
len(bin(int('15')).split('b')[1])

4

In [7]:
assembler = []
for item in subst_a_inst:
    
    if item[0] == '@':
        instruction = '0'
        binary = bin(int(item[1:])).split('b')[1]
        for i in range(15 - len(binary)):
            instruction = instruction + '0'
        instruction = instruction + binary
        
    else:
        instruction = '111'
        cmp = item.split(';')[0]
        
        if len(item.split(';')) > 1:
            i_jump = item.split(';')[1]
        else:
            i_jump = 'null'
        
        if len(cmp.split('=')) > 1:
            i_dest = cmp.split('=')[0]
            i_comp = cmp.split('=')[1]
        else:
            i_dest = 'null'
            i_comp = cmp.split('=')[0]
        
        instruction = '111' + comp[i_comp] + dest[i_dest] + jump[i_jump]
        
    assembler.append(instruction)

In [11]:
hack = open('mult.hack','w+')
for row in assembler:
    hack.write(row + "\n")
hack.close()
original.close()